Note: this solves the 1D advection equation $a_t + u a_x = 0$ using a bunch of different methods.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def set_ics(size,pulse_type='square',method='fou'):
    '''
    Sets up initial conditions for two different pulse types and 1st and 
    2nd order methods (which require) different numbers of ghost zones
    '''
    
    if method == '2oa' or method == '2oa_minmod' or method == '2oa_mcd':
        # 2nd order schemes need 2 ghost zones
        ghost_zones = 2

    else:
        # 1st order need 1 ghost zone
        ghost_zones = 1
    
    # our array has ghost_zones on each side, so 2*ghost_zones
    a = np.zeros(size+2*ghost_zones)
        
        
    if pulse_type=='square':
        
        begin = int(0.35*size)+ghost_zones
        end = int(0.65*size)+ghost_zones
        a[begin:end+1] = 1.0
    
    elif pulse_type=='gaussian':
        
        center = int(0.5*size)+ghost_zones
        sigma = int(0.1*size)
        
        for i in range(size+2*ghost_zones):
            a[i] = np.exp( -float(i-center)**2/float(2.0*sigma**2) )
        
    else:
        print("not implemented!", pulse_type)

    return a
        
def copy_ghost_zones(array,method='fou'):
    
    if method == '2oa' or method == '2oa_minmod' or method == '2oa_mcd':  
        # 2 ghost zones on each side for 2nd order methods
        array[0:2]=array[-4:-2]
        array[-2:]=array[2:4]
         
    else: 
        # 1 ghost zone on each side for 1st order methods
        array[0]=array[-2]
        array[-1]=array[1]
    

In [ ]:
def advect(arr,u,dt,dx,n_grid,method='fou'):
    '''
    Advection routine: for various 1st and 2nd order methods (including 2nd order with limiters)
    calculate the evolution of a single time step.
    '''
    
    if method=='fou':
        # first-order upwind
        
        if u > 0.0:
            arr[1:n_grid+1] = arr[1:n_grid+1] - u*(dt/dx)*(arr[1:n_grid+1]-arr[0:n_grid])           
        else:
            arr[1:n_grid+1] = arr[1:n_grid+1] - u*(dt/dx)*(arr[2:n_grid+2]-arr[1:n_grid+1])           
        
    elif method=='ftcs':
        # forward time center space (warning: garbage results ahead!)
        
        arr[1:n_grid+1] = arr[1:n_grid+1] - u*(dt/dx)*(arr[2:n_grid+2]-arr[0:n_grid])

    elif method=='2oa':
        # 2nd order advection: remember, two ghost zones!
        
        if u > 0.0:
            # use left solution for Riemann problem: plus signs.  Need cell i, i-1
            
            # i+1/2 uses values at cell i, and then i+1,i-1 for da/dx
            dadx = (arr[3:n_grid+3] - arr[1:n_grid+1])/(2.0*dx)
            aplus = arr[2:n_grid+2] + (dx/2.0)*(1-dt*u/dx)*dadx
            
            # i-1/2 uses values at cell i-1, and then i, i-2 for da/dx
            dadx = (arr[2:n_grid+2] - arr[0:n_grid+0])/(2.0*dx)
            aminus = arr[1:n_grid+1] + (dx/2.0)*(1-dt*u/dx)*dadx
            
        else:
            # use right solution for Riemann problem: minus signs.  Need cell i, i+1
            
            # i+1/2 uses values at cell i+1, and then i+2,i for da/dx
            dadx = (arr[4:n_grid+4] - arr[2:n_grid+2])/(2.0*dx)
            aplus = arr[3:n_grid+3] - (dx/2.0)*(1+dt*u/dx)*dadx
            
            # i-1/2 uses values at cell i, and then i+1, i-1 for da/dx
            dadx = (arr[3:n_grid+3] - arr[1:n_grid+1])/(2.0*dx)
            aminus = arr[2:n_grid+2] - (dx/2.0)*(1+dt*u/dx)*dadx
            
        arr[2:n_grid+2] = arr[2:n_grid+2] - u*(dt/dx)*(aplus-aminus)

    elif method=='2oa_minmod':
        # 2nd order advection: remember, two ghost zones!
        # this version uses the minmod limiter
        
        if u > 0.0:
            # use left solution for Riemann problem: plus signs.  Need cell i, i-1
            
            # i+1/2 uses values at cell i, and then i+1,i-1 for da/dx
            dadx = minmod(arr[1:-1],dx)
            aplus = arr[2:n_grid+2] + (dx/2.0)*(1-dt*u/dx)*dadx
            
            # i-1/2 uses values at cell i-1, and then i, i-2 for da/dx
            dadx = minmod(arr[0:-2],dx)
            aminus = arr[1:n_grid+1] + (dx/2.0)*(1-dt*u/dx)*dadx
            
        else:
            # use right solution for Riemann problem: minus signs.  Need cell i, i+1
            
            # i+1/2 uses values at cell i+1, and then i+2,i for da/dx
            dadx = minmod(arr[2:],dx)
            aplus = arr[3:n_grid+3] - (dx/2.0)*(1+dt*u/dx)*dadx
            
            # i-1/2 uses values at cell i, and then i+1, i-1 for da/dx
            dadx = minmod(arr[1:-1],dx)
            aminus = arr[2:n_grid+2] - (dx/2.0)*(1+dt*u/dx)*dadx
            
        arr[2:n_grid+2] = arr[2:n_grid+2] - u*(dt/dx)*(aplus-aminus)

    elif method=='2oa_mcd':
        # 2nd order advection: remember, two ghost zones!
        # this version uses the monotonized central difference limiter
        
        if u > 0.0:
            # use left solution for Riemann problem: plus signs.  Need cell i, i-1
            
            # i+1/2 uses values at cell i, and then i+1,i-1 for da/dx
            dadx = mcd(arr[1:-1],dx)
            aplus = arr[2:n_grid+2] + (dx/2.0)*(1-dt*u/dx)*dadx
            
            # i-1/2 uses values at cell i-1, and then i, i-2 for da/dx
            dadx = mcd(arr[0:-2],dx)
            aminus = arr[1:n_grid+1] + (dx/2.0)*(1-dt*u/dx)*dadx
            
        else:
            # use right solution for Riemann problem: minus signs.  Need cell i, i+1
            
            # i+1/2 uses values at cell i+1, and then i+2,i for da/dx
            dadx = mcd(arr[2:],dx)
            aplus = arr[3:n_grid+3] - (dx/2.0)*(1+dt*u/dx)*dadx
            
            # i-1/2 uses values at cell i, and then i+1, i-1 for da/dx
            dadx = mcd(arr[1:-1],dx)
            aminus = arr[2:n_grid+2] - (dx/2.0)*(1+dt*u/dx)*dadx
            
        arr[2:n_grid+2] = arr[2:n_grid+2] - u*(dt/dx)*(aplus-aminus)
        
    else:
        print("not implemented!", method)
    
def minmod(arr,dx):
    # minmod limiter - assume array has 1 extra zone on each side
    
    dadx = np.zeros(arr.size-2)
    
    for i in range(dadx.size):
        # the array indices here are all +1 more than they naively should be b/c of extra zones
        a = arr[i+1]-arr[i]
        b = arr[i+2]-arr[i+1]
        
        if np.abs(a) < np.abs(b) and a*b > 0:
            
            dadx[i] = a/dx

        elif np.abs(b) < np.abs(a) and a*b > 0:
        
            dadx[i] = b/dx

        else:
            dadx[i] = 0.0
            
    return dadx

def mcd(arr,dx):
    # monotonized central difference limiter (MCD) - assume array has 1 extra zone on each side
    
    dadx = np.zeros(arr.size-2)
    
    for i in range(dadx.size):
        # the array indices here are all +1 more than they naively should be b/c of extra zones
        a = arr[i+1]-arr[i]
        b = arr[i+2]-arr[i+1]
        c = arr[i+2]-arr[i]
        
        if a*b > 0.0:
            dadx[i] = np.fmin( np.fabs(c)/2.0, np.fmin( 2.0*np.fabs(b), 2.0*np.fabs(a) ))*np.sign(c)/dx
            
        else:
            dadx[i] = 0.0
            
    return dadx


In [ ]:
# here's the cell where stuff actually happens!

# user sets this stuff!
n_grid = 100  # number of grid cells
u = 0.1   # speed (can be + or -)
C = 0.01
t_stop = 1.0/np.abs(u)
adv_method = '2oa_minmod'  # many choices - see above.
pulse='square'  # gaussian or square

# leave this alone.
dx = 1.0/n_grid
dt = C * dx/np.abs(u)

# make ICs and plot them out.
avals = set_ics(n_grid,pulse_type=pulse,method=adv_method)
plt.plot(avals,'b-',linewidth=2)
plt.ylim(-0.1,1.1)
plt.xlim(0,n_grid+1)

# loop until we're done
t=0
while t < t_stop:
    
    advect(avals,u,dt,dx,n_grid,method=adv_method)
    copy_ghost_zones(avals,method=adv_method)
    t += dt
    
# plot out final state so we can admire the carnage.
plt.plot(avals,'r-',linewidth=2)